In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 定义简单的两层神经网络
class RZDecoderNet(nn.Module):
    def __init__(self, input_size):
        super(RZDecoderNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# 从文件中加载数据
def load_data_from_file(file_path, bit_length):
    inputs = []
    outputs = []
    with open(file_path, 'r') as f:
        for line in f:
            values = [float(x) for x in line.strip().split()]
            inputs.append(values[:bit_length])  # 前 bit_length 个值为输入信号
            outputs.append(values[bit_length])  # 最后一个值为输出标签

    inputs = torch.tensor(inputs, dtype=torch.float32)
    outputs = torch.tensor(outputs, dtype=torch.float32).unsqueeze(1)  # 添加维度以匹配网络输出
    return inputs, outputs

def load_data_from_file_exp(file_path, bit_length):
    inputs = []
    outputs = []
    with open(file_path, 'r') as f:
        for line in f:
            values = [float(x) for x in line.strip().split()]
            inputs.append(values[:bit_length])  # 前 bit_length 个值为输入信号

    inputs = torch.tensor(inputs, dtype=torch.float32)
    outputs = torch.tensor(outputs, dtype=torch.float32).unsqueeze(1)  # 添加维度以匹配网络输出
    return inputs, outputs

# 训练 RZ 解码网络
def train_rz_decoder(model, train_loader, num_epochs=20, learning_rate=0.01):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# 使用加载的模型进行预测
def predict_with_model(model, inputs):
    with torch.no_grad():  # 禁用梯度计算
        outputs = model(inputs)
        predictions = (outputs >= 0.5).float()  # 阈值设为 0.5，输出 0 或 1
    return predictions

# 参数
bit_length = 2  # 每个比特的长度为 1

# 文件路径
train_file_path = r'E:\gitprogram\data\testdata\train.txt'
test_file_path = r'E:\gitprogram\data\testdata\test.txt'
exp_file_path = r'E:\gitprogram\data\testdata\exp.txt'

# 从文件加载训练和测试数据
train_inputs, train_labels = load_data_from_file(train_file_path, bit_length)
test_inputs, test_labels = load_data_from_file(test_file_path, bit_length)

# 创建数据加载器
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

# 初始化网络
input_size = bit_length
model = RZDecoderNet(input_size)

# 训练网络
train_rz_decoder(model, train_loader, num_epochs=1000, learning_rate=0.01)

# 保存训练后的模型
torch.save(model.state_dict(), 'trained_model.pth')



In [ ]:
# 加载已训练的模型
model.load_state_dict(torch.load('trained_model.pth'))
model.eval()  # 设置为评估模式

# 读取实验数据
exp_inputs, _ = load_data_from_file_exp(exp_file_path, bit_length)

# 进行预测
predicted_outputs = predict_with_model(model, exp_inputs)

# 输出预测结果
print("Predicted outputs:", predicted_outputs.squeeze().int())


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os

# 定义简单的两层神经网络
class RZDecoderNet(nn.Module):
    def __init__(self, input_size):
        super(RZDecoderNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# 从文件中加载数据
def load_data_from_file(file_path, bit_length):
    inputs = []
    outputs = []
    with open(file_path, 'r') as f:
        for line in f:
            values = [float(x) for x in line.strip().split()]
            inputs.append(values[:bit_length])  # 前 bit_length 个值为输入信号
            outputs.append(values[bit_length])  # 最后一个值为输出标签

    inputs = torch.tensor(inputs, dtype=torch.float32)
    outputs = torch.tensor(outputs, dtype=torch.float32).unsqueeze(1)  # 添加维度以匹配网络输出
    return inputs, outputs

def load_data_from_file_exp(file_path, bit_length):
    inputs = []
    outputs = []
    with open(file_path, 'r') as f:
        for line in f:
            values = [float(x) for x in line.strip().split()]
            inputs.append(values[:bit_length])  # 前 bit_length 个值为输入信号

    inputs = torch.tensor(inputs, dtype=torch.float32)
    outputs = torch.tensor(outputs, dtype=torch.float32).unsqueeze(1)  # 添加维度以匹配网络输出
    return inputs, outputs

# 训练 RZ 解码网络
def train_rz_decoder(model, train_loader, val_loader, num_epochs=20, learning_rate=0.01, model_save_path='best_model.pth'):
    criterion = nn.BCELoss()  # 二元交叉熵损失
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')  # 最优验证损失初始化为正无穷
    patience = 5  # 如果验证损失在连续 5 个 epoch 中没有下降，则停止训练
    counter = 0  # 用于计算验证损失没有下降的次数

    for epoch in range(num_epochs):
        model.train()  # 进入训练模式
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 在每个 epoch 结束后，评估模型在验证集上的表现
        val_loss = evaluate_model(model, val_loader, criterion)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}")

        # 保存最优模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0  # 重置计数器
            torch.save(model.state_dict(), model_save_path)  # 保存当前模型
            print(f"Best model saved with validation loss: {best_val_loss:.4f}")
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping due to no improvement in validation loss.")
                break

# 验证模型
def evaluate_model(model, val_loader, criterion):
    model.eval()  # 设置为评估模式
    total_loss = 0
    with torch.no_grad():  # 禁用梯度计算
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    return total_loss / len(val_loader)

# 使用加载的模型进行预测
def predict_with_model(model, inputs):
    model.eval()  # 设置为评估模式
    with torch.no_grad():  # 禁用梯度计算
        outputs = model(inputs)
        predictions = (outputs >= 0.5).float()  # 阈值设为 0.5，输出 0 或 1
    return predictions

# 参数
bit_length = 2  # 每个比特的长度为 1

# 文件路径
train_file_path = r'E:\gitprogram\data\testdata\train.txt'
test_file_path = r'E:\gitprogram\data\testdata\test.txt'
exp_file_path = r'E:\gitprogram\data\testdata\exp.txt'

# 从文件加载训练和测试数据
train_inputs, train_labels = load_data_from_file(train_file_path, bit_length)
test_inputs, test_labels = load_data_from_file(test_file_path, bit_length)

# 创建数据加载器
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

# 创建验证数据加载器
val_dataset = TensorDataset(test_inputs, test_labels)  # 将测试数据作为验证集
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

# 初始化网络
input_size = bit_length
model = RZDecoderNet(input_size)

# 训练网络并保存最优模型
train_rz_decoder(model, train_loader, val_loader, num_epochs=2000, learning_rate=0.01, model_save_path='best_model.pth')

# 加载最优模型
model.load_state_dict(torch.load('best_model.pth'))
model.eval()  # 设置为评估模式

# 读取实验数据
exp_inputs, _ = load_data_from_file_exp(exp_file_path, bit_length)

# 进行预测
predicted_outputs = predict_with_model(model, exp_inputs)

# 输出预测结果
print("Predicted outputs:", predicted_outputs.squeeze().int())
